## Import Packages
* gym - collection of environments for reinforcement learning algorithms.
* numpy - package for scientific computing with Python. 
* random - implements pseudo-random number generators for various distributions.
* math -  mathematical functions defined by the C standard.

In [ ]:
import gym
import numpy as np
import math

## Define the environment
A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The system is controlled by applying a force of +1 or -1 to the cart. The pole starts upright, and the goal is to prevent it from falling over. A reward of +1 is provided for every timestep that the pole remains upright. The episode ends when the pole is more than 15 degrees from vertical, or the cart moves more than 2.4 units from the center.


In [ ]:
env = gym.make('CartPole-v0')

## Define the environment related constants.
*  Every environment comes with an action_space and an observation_space. These attributes are of type Space, and they describe the format of valid actions and observations. The Discrete space allows a fixed range of non-negative numbers, so in this case valid actions are either 0 or 1. The Box space represents an n-dimensional box, so valid observations will be an array of 4 numbers.
* Define the following constants:
   * Number of discrete states (bucket) per state dimension :position,velocity,angle between the pole and the vertical axis and the angular velocity. 
   * Number of discrete actions (left, right) 
   * Bounds for each discrete state
* Define the minimum exploration and learning rate values.
   

In [ ]:
env.action_space.n

In [ ]:
env.observation_space

In [ ]:
env.observation_space.low 

In [ ]:
env.observation_space.high

In [ ]:
NUM_BUCKETS = (1, 1, 6, 3)

In [ ]:
NUM_ACTIONS = env.action_space.n

In [ ]:
STATE_BOUNDS = list(zip(env.observation_space.low, env.observation_space.high))

In [ ]:
STATE_BOUNDS[1] = [-0.5, 0.5]

In [ ]:
STATE_BOUNDS[3] = [-math.radians(50), math.radians(50)]

In [ ]:
STATE_BOUNDS

## Create Q-Table
* Create a Q-Table for each state-action pair
* Define the learning related constants:
* The learning rate must decay but not too fast. The conditions for convergence are the following:
    *sum(alpha(t), 1, inf) = inf
    *sum(alpha(t)^2, 1, inf) < inf
    * Something like alpha = k/(k+t) can work well.

In [ ]:
q_table = np.zeros(NUM_BUCKETS + (NUM_ACTIONS,))

In [ ]:
q_table.shape

In [ ]:
print(q_table)

In [ ]:
EXPLORE_RATE_MIN = 0.01

In [ ]:
LEARNING_RATE_MIN = 0.1

In [ ]:
def get_explore_rate(t):
    return max(EXPLORE_RATE_MIN, min(1, 1.0 - math.log10((t+1)/25)))

In [ ]:
def get_learning_rate(t):
    return max(LEARNING_RATE_MIN, min(0.5, 1.0 - math.log10((t+1)/25)))

## Define method for selecting an action
* Select a random action or the action with the highest q based on explore rate

In [ ]:
def select_action(state, explore_rate):
    if random.random() < explore_rate:
        action = env.action_space.sample()
    else:
        action = np.argmax(q_table[state])
    return action

## Define method to define states
* Discretize the continuous dimensions to a number of buckets. 
* Map the state bounds to the bucket array

In [ ]:
def state_to_bucket(state):
    
    bucket_indices = []
    
    for i in range(len(state)):
        if state[i] <= STATE_BOUNDS[i][0]:
            bucket_index = 0
            
        elif state[i] >= STATE_BOUNDS[i][1]:
            bucket_index = NUM_BUCKETS[i] - 1
            
        else:
            bound_width = STATE_BOUNDS[i][1] - STATE_BOUNDS[i][0]
            
            offset = (NUM_BUCKETS[i] - 1) * STATE_BOUNDS[i][0] / bound_width
            scaling = (NUM_BUCKETS[i] - 1) / bound_width
            
            bucket_index = int(round(scaling * state[i] - offset))
            
        bucket_indices.append(bucket_index)
    return tuple(bucket_indices)

## Define method for simulation
* Get the initial learning and explore rates
* Intialise discount factor and number of streaks
* Train over 1000 episodes. In each episode, do the following:
* Start the process using the reset method
* Get the initial state by passing in the current observations about the environment
* Iterate over 250 timesteps:
    * Invoke the render method to visualise the episode.
    * Execute a selected action using the step emthod and read the values of reward, observation, done and info.
    * Identify the resultant state from the observations.
    * Identify the state with the highest possible reward from the q table.
    * Compute the q value and update the q table : Q[s, a] = Q[s, a] + alpha*(R + gamma*Max[Q(s’, A)] - Q[s, a])
    
    * Set the new state as current state.
    * If done equals true, episode has terminated.Reset the environment again.
    * The agent wouldn't need more than 200 timesteps to train. Increment the no_streaks value so we can terminate training in 120 episodes, each of no more than 200 timesteps.
    * Update the parameters so the learning and exploration rates decay as the episodes increase
    

In [ ]:
def simulate():

    learning_rate = get_learning_rate(0)
    explore_rate = get_explore_rate(0)

    discount_factor = 0.99  
    num_streaks = 0

    for episode in range(1000):
        
        observ = env.reset()
    
        state_0 = state_to_bucket(observ)

        for t in range(250):

            env.render()
            
            action = select_action(state_0, explore_rate)
            
            observ, reward, done, _ = env.step(action)

            state = state_to_bucket(observ)
            
            best_q = np.amax(q_table[state])
            
            q_table[state_0 + (action,)] +=\
                learning_rate * (reward + discount_factor*(best_q) - q_table[state_0 + (action,)])

            state_0 = state

            print("\nEpisode = %d" % episode)
            print("t = %d" % t)
            print("Action: %d" % action)
            print("State: %s" % str(state))
            print("Reward: %f" % reward)
            print("Best Q: %f" % best_q)
            print("Explore rate: %f" % explore_rate)
            print("Learning rate: %f" % learning_rate)
            print("Streaks: %d" % num_streaks)

            print("")

            if done:
                print("Episode %d finished after %f time steps" % (episode, t))
                if (t >= 199):
                    num_streaks += 1
                else:
                    num_streaks = 0
                break

        if num_streaks > 120:
            break
      
        explore_rate = get_explore_rate(episode)
        learning_rate = get_learning_rate(episode)

## Train the agent
* Invoke the simulate method
* Close the rendered environment.
* Check the values in q talbe.

In [ ]:
simulate()

In [ ]:
env.close()

In [ ]:
print(q_table)